*Master IASD, PSL - 2023/2024 - O. Cappé, February 2024*

# Retail Store Management

**This is your INDIVIDUAL homework that needs to be returned by March 4, 2024, at the latest, as a functional completed Python notebook file. Late submissions will be applied a penalty.** Please print your name here and be sure to name your file <code>YourFirstName-YOURLASTNAME-retail_store.ipynb</code> and to send it by email to <olivier.cappe@ens.fr> before the deadline.

We consider, the retail store management model seen in the first course.

You own a bike store. During week $t$, the demand is $D_t$ units, which we may assume to be $\operatorname{Poisson}(d)$ distributed, independently of the past. On Monday morning you may choose to command $A_t$ additional units that are delivered immediately before the shop opens. For each week,

- Maintenance Cost: $h$ per unit left in your stock from previous week
- Ordering Cost: $c$ per ordered unit
- Sales Profit: $f$ per sold unit

With the following constraints

- Your warehouse has a maximal capacity of $m$ unit (any additional bike gets stolen)
- You cannot sell bikes that you don’t have in stock

We will consider that $\mathcal{A}=\mathcal{S}=\{0,\dots,m\}$ and the MDP evolves according to

- $D_t \sim \operatorname{Poisson}(d)$
- $X_t = -h S_t -c A_t + f \min(D_t, S_t + A_t, m)$
- $S_{t+1} = \max(0, \min(S_t+A_t,m)-D_t)$

In [1]:
# Please REFRAIN from importing any additional module
import math
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt

## The Retail Store Environment

The <code>RetailStore</code> class defines the environnement, providing the basic functions for interacting with the system (<code>env.reset</code> and <code>env.step</code>) and for computing basic parameter-dependent quantities (reward and transition functions, value function of a policy).

In [2]:
# Utility functions for the Retail store environment (do NOT modify this code block)

class RetailStore:


    def __init__(self, m, h, c, f, d):
        self.m = m  # Stock capacity
        self.h = h  # Maintenance cost per unit
        self.c = c  # Buying price per unit
        self.f = f  # Selling price per unit
        self.d = d  # Weekly average demand


def reset(self, state):
    """ Restarts the environment at time 0 in specified state. """
    self.state = state
    self.time = 0


def step(self, action):
    """ Given the action, performs one call to the environment and return the reward. """
    demand = poisson.rvs(self.d)
    reward = -self.h * self.state - self.c * action + self.f * min([demand, self.state + action, self.m])
    # Update the time and state variables
    self.time += 1
    self.state = max([min([self.state + action, self.m]) - demand, 0])
    return reward


def reward_function(self):
    """ Computes the action-depend reward function r(s,a). """
    r = np.zeros((self.m + 1, self.m + 1))
    for s in range(self.m + 1):
        for a in range(self.m + 1):
            # Note: computing the expectation of the truncated Poisson distribution using the survival function
            r[s, a] = -self.h * s - self.c * a + self.f * sum(
                poisson.sf(np.linspace(0, min(s + a, self.m) - 1, num=min(s + a, self.m)), self.d))
    return r


def transition_function(self):
    """ Computes the action-depend transition probabilities p(s,a,s'). """
    p = np.zeros((self.m + 1, self.m + 1, self.m + 1))
    for s in range(self.m + 1):
        for a in range(self.m + 1):
            for i in range(min(s + a, self.m)):
                p[s, a, min(s + a, self.m) - i] = poisson.pmf(i, self.d)
            p[s, a, 0] = poisson.sf(min(s + a, self.m) - 1, self.d)
    return p


def reward_policy(self, pi):
    """ Computes the reward function r_pi(s) associated with a policy. """
    r = self.reward_function()
    r_pi = np.sum(np.multiply(r, pi), axis=1)
    return r_pi


def transition_policy(self, pi):
    """ Computes the transition probabilities p_pi(s,s') associated with a policy. """
    p = self.transition_function()
    p_pi = np.zeros((self.m + 1, self.m + 1))
    for s in range(self.m + 1):
        p_pi[s, :] = np.matmul(np.transpose(pi[s, :]), p[s, :, :])
    return p_pi


def value_policy(self, pi, gamma):
    """ Computes the value function of a policy, with discount gamma (using matrix inversion). """
    r_pi = self.reward_policy(pi)
    p_pi = self.transition_policy(pi)
    v_pi = np.linalg.solve(np.eye(self.m + 1) - gamma * p_pi, r_pi)
    return v_pi

## Experiments

### Q1. Simulation of Fixed Ordering Policies

We will consider a small-size model in which $[m, h, c, f, d, \gamma] = [9, 0.1, 0.5, 1, 4, 0.875]$ **(Comment these choice of parameters).** By simulating trajectories from the model **(discuss what length is necessary)** get an empirical idea of **how well fixed-ordering (i.e. ordering always the same quantity of goods) perform?** Use the <code>env.value_policy</code> to **plot the value functions of the fixed-ordering policies.** **What is your interpretation of the results? Do you think that there are better policies?**

In [3]:
# Instantiate the retail store environment with the selected parameters 
[m, h, c, f, d, gamma] = [9, 0.1, 0.5, 1, 4, 0.875]
env = RetailStore(m, h, c, f, d)

# Just an example of simulating a short trajectory and computing the cumulated reward  
env.reset(m)
n = 12
w = 1
v = 0
for _ in range(n):
    x = env.step(3)
    print(env.time - 1, ': ', '{:.1f}'.format(x), ' ->', env.state)
    v += w * x
    w *= gamma
print('Discounted reward:', '{:.1f}'.format(v))

# --- Your answer here ---

0 :  3.6  -> 3
1 :  3.2  -> 1
2 :  2.4  -> 0
3 :  1.5  -> 0
4 :  -0.5  -> 2
5 :  3.3  -> 0
6 :  -0.5  -> 2
7 :  3.3  -> 0
8 :  0.5  -> 1
9 :  1.4  -> 1
10 :  2.4  -> 0
11 :  -1.5  -> 3
Discounted reward: 12.6


--- _Your answer here_ ---

### Q2. Computing the Optimal Policy
Obtain the optimal policy by **implementing the Policy Iteration algorithm** (use the <code>env.value_policy</code> to compute the value function). **How do you know that it has converged?** **What does the optimal policy do?** **Comment the form of the value function.**

In [4]:
# --- Your answer here ---
def policy_iteration(env, gamma, max_iter=1000, tol=1e-6):
    m = env.m
    pi = np.zeros((m + 1, m + 1))
    for i in range(m + 1):
        pi[i, i] = 1
    for _ in range(max_iter):
        v = env.value_policy(pi, gamma)
        q = env.reward_function() + gamma * np.matmul(env.transition_function(), v)
        pi_new = np.zeros((m + 1, m + 1))
        for s in range(m + 1):
            pi_new[s, np.argmax(q[s, :])] = 1
        if np.linalg.norm(pi_new - pi) < tol:
            break
        pi = pi_new
    return pi, v


pi, v = policy_iteration(env, gamma)
print(pi)
print(v)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[12.66641819 13.06641819 13.46641819 13.86641819 14.26641819 14.66641819
 15.05591511 15.37589753 15.65065105 15.89190991]


-- _Your answer_ here ---

### Q3. Q Learning
**Implement the asynchronous Q-Learning algorithm using the purely random policy (all actions selected uniformly) as the exploration policy** (note: this requires only about 10 lines of code). **Plot the convergence of the algorithm** both in terms of convergence of the state-action value table $Q_t$ and of convergence of the value function of the associated greedy policy $\pi_{t+}(s) = \arg\max_a Q_t(s,a)$. Use a single trajectory of length $n = 10,000$ and **test different schemes of decrease** of the learning rate (following the course guidelines). 

In [5]:
# Your answer here

### Q4. Policy Gradient

We will now consider using policy gradient from a set of simulated trajectories using the REINFORCE formula to approximate the gradient of the value function. To do so, we consider a log-linear parameterization of the policy and provide the two utility functions below.

<code>policy_features</code> Computes a $2(m+1)$--dimensional feature vector $\phi(s,a)$ corresponding to the state-action pair $(s,a)$.

<code>policy_choice</code> Computes the vector $\pi_\theta(s,:)$ of action probabilities using the softmax operator:
$$
    \pi_\theta(s,a) = \frac{\exp\left(\theta^T \phi(s,a)\right)}{\sum_{a'=0}^m \exp\left(\theta^T \phi(s,a')\right)}
$$
You should check from the code that the matrix $(\theta^T \phi(s,a))_{0\leq s,a \leq n}$ is a weighted sum of the $m+1$ fixed ordering policies and of the $m+1$ threshold policies (which you should have met already...)

In [6]:
# Policy gradient utility functions (do NOT modify this code block)

def policy_features(s, a, m):
    """Returns the feature vector corresponding to state (s,a)."""
    f = np.zeros(2 * (m + 1))
    # Index of the fixed ordering policy that is non zero in (s,a)
    f[a] = 1
    # Indices of the threshold policies that are non zero in (s,a)
    if (s + a <= m):
        f[(m + 1) + s + a] = 1
    if (a == 0):
        for i in range(0, s + 1):
            f[(m + 1) + i] = 1
    return f


def policy_choice(s, m, theta):
    """Returns the vector of action probabilities pi(s,:) corresponding to state s and parameter theta."""
    pi = np.zeros(m + 1)
    # Note : Computation in log to avoid numerical underflows
    for a in range(m + 1):
        pi[a] = np.dot(policy_features(s, a, m), theta)
    pi = pi - np.max(pi)
    pi = np.exp(pi) / np.sum(np.exp(pi))
    return pi

**Code a function** <code>policy_gradient</code> that computes the gradient $\nabla_\theta \log\pi_\theta(s,a)$ of $\pi_\theta(s,a)$. First **write in the text block below the LaTeX formula implemented by the function, explaining how you obtain this formula.**

--- _Your answer here_ ---

In [7]:
# --- Your answer here ---

**Implement the policy gradient algorithm** approximating the gradients using the REINFORCE formula
$$
  \sum_{t=0}^{n} \gamma^t \left(\sum_{i=0}^{n} \gamma^i X_{t+i} \right) \nabla_\theta \log \pi_\theta(S_t, A_t)
$$
computed on trajectories of length 35 started from a random initial state and using 200 iterations of SGD updates. To do so, complete the code template provided below. **Monitor the convergence of the algorithm by plotting the difference between the mean of the optimal value function and the mean of the value functions corresponding to successive values of $\theta$ (explain why one considers the mean).**

In [8]:
# --- Edit this block ---
n = 35
nb_iter = 200

theta = np.zeros(2 * (m + 1))
for i_iter in range(nb_iter):
    # Compute the REINFORCE approximation of the gradient of the value function from a run of length n
    # of the MDP initialized t a random state and using the policy corresponding to the current value of theta

    grad = np.zeros(2 * (m + 1))  # --- Something better than this is needed here! ---

    # Do a step of SGD update on theta
    theta += 0.1 * np.power(1 + i_iter, -0.6) * grad